In [1]:
%matplotlib inline
#import proplot as pplt # plotting
import numpy as np # might be useful
import pandas as pd # for reading the csv files
import datetime as dt # for date handling
#import PyCO2SYS as pyco2 # Carbon Chemistry
#import BG.readDat as rd # reading the files
from datetime import datetime
from os import listdir as listdir
#import cftime as cft
import xarray as xr
#from matplotlib import pyplot as plt
#import cartopy.feature as cfeature
#import scipy as scipy
import seawater as sw
import zipfile as zip # to read all files contained in a zip archive
# own functions:
from gridcellfinder import gridcellfinder as gridcellfinder


Script to convert the data from the Arctic Net cruises into netcdf's with NEMO-depth. 
Nemo_ nav_lon and nav_lat?!!!!

In [2]:
# Open the file containing the NAA depth info and prepare
ncds = xr.open_dataset('DATA/NAA_Area_Volume.nc') # this is the updated file with corrected depth cut-offs
z = [0]
zc  = list(ncds.z_cuts.values)
zcu = z + zc
deptht = ncds.deptht.values.round(5)


In [3]:
#define some functions to reduce copypasta
def metacleaner(meta): 
  '''
  goes through the meta dataframe and converts it into a cleaned up dictionary for later use
  
  '''
  mdict = {}
  for i in range(0,len(meta)):
    #print(i)
    fu = str(meta.iloc[i,0]) #nice
    if len(fu) > 2 :
      fu = fu.split(':', 1) 
      mdict[fu[0][2:]] = fu[1] # fill the dictionary
      #print(fu[0])
    else: 
      pass
      
  # Task: find the appropriate gridcells on the NEMO-grid
  mdict['Latitude']  = float(mdict['Initial_Latitude [deg]'])
  mdict['Longitude'] = float(mdict['Initial_Longitude [deg]'])
  mdict['lons'] = round(mdict['Longitude'], 2) 
  mdict['lats'] = round(mdict['Latitude'], 2)
  mdict['lonmin'] = mdict['lons'] - 0.1 #0.1 in degrees is appox 10 km -> i read in  Roberts(?) 2022 paper, they did the same
  mdict['lonmax'] = mdict['lons'] + 0.1
  mdict['latmin'] = mdict['lats'] - 0.1
  mdict['latmax'] = mdict['lats'] + 0.1
  #find the NAA-gridcell
  x,y =gridcellfinder(mdict, ncds,0)
  #print(type(x))
  if len(x) ==0:
    X = [999]
  else: 
    X = x.tolist() 
  if len(y) == 0:
    Y = [999]
  else: 
    Y = y.tolist()   
  mdict['X_NAA'] = str(X)
  mdict['Y_NAA'] = str(Y) 

  # get the Station identifier: 
  ts = pd.to_datetime(mdict['Start_Date_Time [UTC]'])
  year =ts.strftime('%Y')
  month = ts.strftime('%m')
  #assign a unique Station identifyier: YYYY-MM_Source_type_filename-or-other
  mdict['Stat_id'] = year+'-'+month+'_AMS_CDT_' + mdict['Cruise_Number'].replace(" ", "")+'_'+ mdict['Cast_Number  '].replace(" ", "")

  
  return(mdict)




def depth_calc(ds, dct):
  ''' ds = dataset, dct = dictionary'''
  # convert pressure to depth 
  mylist = list(ds)
  # make sure the data is numeric/float
  for i in mylist: 
    ds[i] = pd.to_numeric(ds[i])
  # use Seawater to calculate depth 
  depth = sw.dpth(p = ds['Pres'], lat = dct['Latitude'] )
  ds['Depth'] = depth
    
  return(ds)



In [4]:
def array_cleaner(array):
  ''''renames a bunch of variables to an uniform thing'''''
  fo = list(array)
  if 'PRES' in fo:   array.rename(columns = {'PRES':'Pres' }, inplace = True)
  else: pass
  if 'TE90' in fo: array.rename(columns ={'TE90':'Temp'}, inplace = True)
  else: pass
  if 'FLOR' in fo: array.rename(columns ={'FLOR':'Fluo' }, inplace = True)
  else: pass
  if 'TRAN' in fo: array.rename(columns ={'TRAN':'Trans'}, inplace = True)
  else: pass
  if 'NTRA' in fo: array.rename(columns ={'NTRA':'NO3'}, inplace = True)
  else: pass
  if 'PSAR' in fo: array.rename(columns ={'PSAR':'Par'}, inplace = True)
  else: pass
  if 'SPAR' in fo: array.rename(columns ={'SPAR':'SPar'}, inplace = True)
  else: pass
  if 'PSAL' in fo: array.rename(columns ={'PSAL':'Sal'}, inplace = True)
  else: pass
  if 'OXYM' in fo: array.rename(columns ={'OXYM':'O2'}, inplace = True)
  else: pass
  if 'SIGM' in fo: array.rename(columns ={'SIGM':'Dens'}, inplace = True)
  else: pass
  if 'SPVA' in fo: array.rename(columns ={'SPVA':'Svan'}, inplace = True)            
  else: pass
  if 'svan' in fo: array.rename(columns ={'svan':'Svan'}, inplace = True)
  else: pass
  if 'VAIS' in fo: array.rename(columns ={ 'VAIS':'N2' }, inplace = True)
  else: pass
  if 'SIGT' in fo: array.rename(columns ={ 'SIGT':'Sigt' }, inplace = True)
  else: pass
  if 'POTM' in fo: array.rename(columns ={ 'POTM':'Theta' }, inplace = True)
  else: pass
  if 'SIGP' in fo: array.rename(columns ={ 'SIGP':'Sigthe' }, inplace = True)
  else: pass
  if 'sigthe' in fo: array.rename(columns ={ 'sigthe':'Sigthe' }, inplace = True) 
  else: pass
  if 'FRET' in fo: array.rename(columns ={ 'FRET':'FreezT' }, inplace = True)
  else: pass
  if 'ASAL' in fo: array.rename(columns ={ 'ASAL':'Asal' }, inplace = True)  
  else: pass
  if 'CONT' in fo: array.rename(columns ={ 'CONT':'Cont' }, inplace = True)
  else: pass
  if 'D_CT' in fo: array.rename(columns ={ 'D_CT':'D_ct' }, inplace = True)
  else: pass
  if 'D0CT' in fo: array.rename(columns ={ 'D0CT':'D0ct'}, inplace = True)
  else: pass
  if 'sigt' in fo: array.rename(columns ={'sigt':'Sigt'  }, inplace = True)
  else: pass
  if 'theta' in fo: array.rename(columns ={'theta':'Theta'  }, inplace = True)
  else: pass
  if'CDOM' in fo: array.rename(columns ={'CDOM': 'Cdom' }, inplace = True)
  else: pass 
  if 'FreezT-'in fo: array.rename(columns ={'FreezT-': 'FreezT' }, inplace = True)
  else:pass
  return( array)

  

In [5]:
longnamesdict = {
 'Date_Time': ['', '[UTC]'],
 'original_Latitude':['',  '[deg]'],
 'original_Longitude': ['', '[deg]'],
 #'Sounding [m]':['',  '[m]'],
 'Min_Depth': ['', '[m]'],
 'Max_Depth':['', '[m]'],
 'Pres': ['Sea Pressure (sea surface - 0)', '[decibars]'],
 'Temp': ['Temperature (1990 scale) ', '[deg C]'],
 'Trans': ['Light Transmission ', '[%]'],
 'Fluo': ['Fluorescence ', '[mg/m^3]'],
 'Sal': ['Practical Salinity ', '[psu]'],
 'Dens': ['Density (rho(s, t, p)-1000) ', '[kg/m^3]'],
 'Svan': ['Specific Volume Anomaly ', '[m^3/kg]'],
 'N2': ['Brunt-Vaisala Frequency N2 ', '[s^-2]'],
 'Sigt': ['Sigma-0 ', '[kg/m^3]'],
 'Theta': ['Potential Temperature ', '[deg C]'],
 'Sigthe': ['Sigma-Theta (rho(s, Theta, 0)-1000) ', '[kg/m^3]'],
 'FreezT': ['Freezing Temperature ', '[deg C]'],
 'O2': ['Dissolved Oxygen ', '[ml/l]'],
 'Cdom': ['Fluorescence of CDOM (Colored Dissolved Organic Matter) ', '[mg/m^3]'],
 'NO3': ['Nitrate (NO3-N) Content ', '[mmol/m^3]'],
 'Par': ['Photosynthetic Active Radiation ', '[µeinsteins/s/m^2]'],
 'SPar': ['Surface Photosynthetic Active Radiation ', '[µeinsteins/s/m^2]'],
 'Asal': ['Absolute Salinity TEOS-10 ', '[g/kg]'],
 'Cont': ['Conservative Temperature TEOS-10 ', '[deg C]'],
 'D_ct': ['Density TEOS-10 ((s, t, p) - 1000) ', '[kg/m^3]'],
 'D0ct': ['Density TEOS-10 (s, t, 0) ', '[kg/m^3]']}

def create_array(dat, dct): 
  '''wraper to create the individual arrays'''
  ds = xr.Dataset(dat)#, coords = {'NAA_deptht': dat['NAA_deptht']})
  #ds = xr.Dataset(data_vars=dat,
    #dims=["z", "N"],
  #  coords=dict(deptht=(["z"], dat['NAA_deptht']),
  #              Station=(['N'], dct['Stat_id'])))
  #dat.to_xarray()
  ds= ds.assign_coords(coords ={'Station':  dct['Stat_id']})
  #ds = as_var_attr(ds = ds, newname = 'Cruise_Number', dct = dct, dictname= 'Temp' )
  
  ds= ds.assign(Cruise_Number= dct['Cruise_Number'], 
     Cruise_Name = dct['Cruise_Name'], Cast_Number= dct['Cast_Number  '], Date_Time= dct['Start_Date_Time [UTC]'],  
     original_Latitude= dct['Initial_Latitude [deg]'], original_Longitude= dct['Initial_Longitude [deg]'], 
     Min_Depth=  dct['Min_Depth [m]'], Max_Depth=  dct['Max_Depth [m]'], Latitude=   dct['Latitude'], 
     Longitude =  dct['Longitude'], lons=  dct['lons'], lats=  dct['lats'], lonmin =  dct['lonmin'], lonmax =  dct['lonmax'], 
     latmin =  dct['latmin'], latmax =  dct['latmax'], X_NAA = dct['X_NAA'], Y_NAA = dct['Y_NAA'], 
     Stat_id = dct['Stat_id'])#,
  try: 
    ds = ds.assign(original_Station=  dct['Station '])
  except: 
    try: 
      ds = ds.assign(original_Station=  dct['Station'])   
    except: 
      ds = ds.assign(original_Station = 'No_original_Station')  
  # Assign attributes to the data variables:

  for t in longnamesdict:
    try: 
     ds[t].attrs['long_name'] = longnamesdict[t][0]
     ds[t].attrs['units'] = longnamesdict[t][1]
    except: pass  
  return(ds)

#ds = create_array(dat= data, dct = mdict)
#ds


In [6]:
# get list of files
files = listdir('DATA/OBS/Arctic Net Cruises/WD/')

# Loop over the list of files

for f in files: 
  
  if f == 'csv_READMEs':  # this is a folder, ignore
    pass #print(f) #pass
  elif f == '0501_int.zip': # This dataset is broken
    print(f, 'not workgin')
    pass
  elif f == '0502_int.zip': # This dataset is broken
    print(f, 'not workgin')
    pass
  else: 
    archive = zip.ZipFile('DATA/OBS/Arctic Net Cruises/WD/'+ f)
    #print(f)
    f_list = archive.namelist()
    
    test = pd.read_fwf(archive.open(f_list[0]), encoding = 'latin-1')
    try: 
      dead_rows = test['%'].dropna() # the length of the comment section varies strongly!
      c = len(dead_rows)+1
      data = pd.read_fwf(archive.open(f_list[0]),  encoding='latin-1', skiprows=c,)#archive.read('1405_003.int'
      lendat= len(data)
      meta = pd.read_table(archive.open(f_list[0]), skiprows= list(range(c, lendat+c+1 )), header = None, encoding = 'latin-1')
    except:
      print('This dataset needs to be fixed by hand - no Meta-data')
      print(f_list[0])

    # turn Meta into a dictionary with : as separator, to have a better metadatafile
    # also, get the Longitude/Latidute sorted out with the NEMO-grid
    mdict = metacleaner(meta)  
    #print(list(data))
    data = data.drop(0).reset_index()
    data = data.drop(['index'], axis = 1) 
    #print(list(data))
    # calculate the depth
    data = array_cleaner(data)
    #print(list(data))
    data = depth_calc(ds=data,dct = mdict)
    # use pandas to cut the depth values (compare to Michel_2021.ipynb)
    data['NAA_deptht'] = pd.cut(data['Depth'], bins = zcu , labels =  deptht)
    # take the mean values for each depth bin
    data = data.groupby('NAA_deptht').mean().reset_index()
    data = data.set_index('NAA_deptht')
    # turn data into an array with coordinate Stat_id
    ds = create_array(dat= data, dct = mdict)
 
    #### Loop over the rest of the files ####    
    for j in range(1,len(f_list)):
      test = pd.read_fwf(archive.open(f_list[j]), encoding = 'latin-1')
      try: # the length of the comment section varies strongly!, some dont even have any
        dead_rows = test['%'].dropna() 
        c = len(dead_rows)+1
        data = pd.read_fwf(archive.open(f_list[j]),  encoding='latin-1', skiprows=c,)#archive.read('1405_003.int'
        lendat= len(data)
        meta = pd.read_table(archive.open(f_list[j]), skiprows= list(range(c, lendat+c+1 )), header = None, encoding = 'latin-1')
      except:
        print('This dataset needs to be fixed by hand - no Meta-data')
        print(f_list[j])
        
      mdict = metacleaner(meta)  
      data = data.drop(0).reset_index()
      data = data.drop(['index'], axis = 1) 

      # calculate the depth
      data = array_cleaner(data) # some datasets have different names for the variables.
      data = depth_calc(ds=data,dct = mdict) 

      # use pandas to cut the depth values (compare to Michel_2021.ipynb)
      data['NAA_deptht'] = pd.cut(data['Depth'], bins = zcu , labels =  deptht)

      # take the mean values for each depth bin
      data = data.groupby('NAA_deptht').mean().reset_index()
      data = data.set_index('NAA_deptht')
      # turn data into an array with coordinate Stat_id
      slce = create_array(dat= data, dct = mdict)
        
      ds = xr.concat([ds, slce], dim = 'Station')
      
      
    ds.attrs = { 'script': 'created with Arctic_net_cruises.ipynb -- JLanger',
                'cluster': 'Aeolus - Uvic',
                'creation Date--time': str(datetime.now()), 
                'Data-source': 'Amundsen Sciene, Polar data catalogue' }
    ds.to_netcdf('OUTPUT/AMS/'+ f[:-4]+ '.nc' )
    print('OUTPUT/AMS/'+ f[:-4]+ '.nc written')

print('finished')   

['Pres', 'Temp', 'Trans', 'Fluo', 'Sal', 'Dens', 'Svan', 'N2', 'Sigt', 'Theta', 'sigthe', 'FreezT', 'O2', 'Cdom', 'NO3', 'Par', 'SPar', 'Asal', 'Cont', 'D_ct', 'D0ct']
['Pres', 'Temp', 'Trans', 'Fluo', 'Sal', 'Dens', 'Svan', 'N2', 'Sigt', 'Theta', 'Sigthe', 'FreezT', 'O2', 'Cdom', 'NO3', 'Par', 'SPar', 'Asal', 'Cont', 'D_ct', 'D0ct']
OUTPUT/AMS/1601_int_R1.nc written
['Pres', 'Temp', 'Trans', 'Fluo', 'Sal', 'Dens', 'Svan', 'N2', 'Sigt', 'Theta', 'Sigthe', 'FreezT', 'O2', 'Cdom', 'NO3', 'Par', 'SPar', 'Asal', 'Cont', 'D_ct', 'D0ct']
['Pres', 'Temp', 'Trans', 'Fluo', 'Sal', 'Dens', 'Svan', 'N2', 'Sigt', 'Theta', 'Sigthe', 'FreezT', 'O2', 'Cdom', 'NO3', 'Par', 'SPar', 'Asal', 'Cont', 'D_ct', 'D0ct']
OUTPUT/AMS/1502_int_R4.nc written
['Pres', 'Temp', 'Trans', 'Fluo', 'Sal', 'Dens', 'svan', 'N2', 'sigt', 'theta', 'sigthe', 'FreezT', 'O2', 'CDOM', 'NO3', 'Par', 'SPar', 'ASAL', 'CONT', 'D_CT', 'D0CT']
['Pres', 'Temp', 'Trans', 'Fluo', 'Sal', 'Dens', 'Svan', 'N2', 'Sigt', 'Theta', 'Sigthe', 'F

In [7]:
list(data)

['Pres',
 'Temp',
 'Trans',
 'Fluo',
 'Sal',
 'Dens',
 'Svan',
 'N2',
 'Sigt',
 'Theta',
 'Sigthe',
 'FreezT',
 'O2',
 'pH',
 'NO3',
 'Par',
 'SPar',
 'Depth']